In [3]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use(['science', 'notebook', 'grid'])

In [6]:
#Import the error arrays
err_con = np.loadtxt('C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Prototype 3\\Feedback\\Data\\Front Concentric Error.txt')
err_ecc = np.loadtxt('C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Prototype 3\\Feedback\\Data\\Front Eccentric Error.txt')

#Calculate the mean and standard deviation of the errors for each feature.
err_con_mean = np.round(np.mean(err_con, axis=1),3)
err_con_std = np.round(np.std(err_con, axis=1),3)
err_ecc_mean = np.round(np.mean(err_ecc, axis=1),3)
err_ecc_std = np.round(np.std(err_ecc, axis=1),3)

# change the angular values to degrees
err_con_mean[:5] = np.rad2deg(err_con_mean[:5])
err_con_std[:5] = np.rad2deg(err_con_std[:5])
err_ecc_mean[:5] = np.rad2deg(err_ecc_mean[:5])
err_ecc_std[:5] = np.rad2deg(err_ecc_std[:5])

err_con_std,err_ecc_std

(array([ 3.83881723,  9.74028252, 13.12073351,  3.09397209]),
 array([ 3.55233833,  8.65166271, 12.83425461,  2.86478898]))

In [7]:
import sys
sys.path.append("C:\\Users\\Cian\\1 FYP Code\\Code_with_git\\Functions")
from Functions_Squat import signed_error

In [4]:
con_err, con_interp, corr_con, ecc_err, ecc_interp, corr_ecc = signed_error(100, "C:\\Users\\Cian\\Downloads\\front.mp4")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[ 0.6         1.73333333  2.66666667  3.6         5.06666667  6.26666667
  7.8         9.26666667 11.2       ]
Front


In [15]:
means = []
for i in range(len(con_err)):
    splits = np.array_split(con_err[i,:,5], 3)
    means.append([np.mean(splits[j]) for j in range(3)])
means = np.array(means).ravel()

In [16]:
means

array([ 0.07128114,  0.10820279,  0.04746332, -0.55756989, -0.4960694 ,
       -0.17545466, -0.46561928, -0.45812805, -0.27346223])

In [17]:
labels = np.array([0,0,0,-0.5,-0.5,0,-0.5,-1,0])
np.savetxt("C:\\Users\\Cian\\Downloads\\data_t", means)
np.savetxt("C:\\Users\\Cian\\Downloads\\data_l", labels)

In [7]:
f=0
#Acquire the values for defining the "correct" membership function
corr = np.array([err_ecc_mean[f]-3.5*err_ecc_std[f],err_ecc_mean[f]-2*err_ecc_std[f], err_ecc_mean[f]+2*err_ecc_std[f], err_ecc_mean[f]+3.5*err_ecc_std[f]])
med_neg = np.array([corr[0]-3*err_ecc_std[f], corr[0]-2*err_ecc_std[f], corr[0], corr[1]])
med_pos = np.array([corr[-2], corr[-1], corr[-1]+2*err_ecc_std[f], corr[-1]+3*err_ecc_std[f]])
lar_neg = np.array([-100, -100, med_neg[0], med_neg[1]])
lar_pos = np.array([med_pos[-2], med_pos[-1],100,100])

#def inputs
beg = ctrl.Antecedent(np.arange(-100,100,0.1),'Beginning')

#def membership functions
beg['Correct'] = fuzz.trapmf(beg.universe, corr)
beg['Medium_neg'] = fuzz.trapmf(beg.universe, med_neg)
beg['Medium_pos'] = fuzz.trapmf(beg.universe, med_pos)
beg['Large_neg'] = fuzz.trapmf(beg.universe, lar_neg)
beg['Large_pos'] = fuzz.trapmf(beg.universe, lar_pos)


#def outputs
out_beg = ctrl.Consequent(np.arange(-1.5, 1.51, 0.01), 'output')

out_beg['Incorrect Negative'] = fuzz.trapmf(out_beg.universe,[-1.5,-1.2,-0.8,-0.5])
out_beg['Medium Negative'] = fuzz.trapmf(out_beg.universe,[-0.8,-0.6,-0.4,-0.2])
out_beg['Correct'] = fuzz.trapmf(out_beg.universe,[-0.5,-0.2,0.2,0.5])
out_beg['Medium Positive'] = fuzz.trapmf(out_beg.universe,[0.2,0.4,0.6,0.8])
out_beg['Incorrect Positive'] = fuzz.trapmf(out_beg.universe,[0.5,0.8,1.2,1.5])


#def rules
rules_beg = []

rules_beg.append(ctrl.Rule(beg['Correct'], out_beg['Correct']))
rules_beg.append(ctrl.Rule(beg['Medium_neg'], out_beg['Medium Negative']))
rules_beg.append(ctrl.Rule(beg['Medium_pos'], out_beg['Medium Positive']))
rules_beg.append(ctrl.Rule(beg['Large_neg'], out_beg['Incorrect Negative']))
rules_beg.append(ctrl.Rule(beg['Large_pos'], out_beg['Incorrect Positive']))

#def system
sys_beg = ctrl.ControlSystem(rules_beg)

In [8]:
#simulate beginning
sim_beg = ctrl.ControlSystemSimulation(sys_beg)
sim_beg.input['Beginning'] = means[0]
sim_beg.compute()
print(np.round(sim_beg.output['output'],3))

#simulate beginning
sim_beg = ctrl.ControlSystemSimulation(sys_beg)
sim_beg.input['Beginning'] = means[1]
sim_beg.compute()
print(np.round(sim_beg.output['output'],3))

#simulate beginning
sim_beg = ctrl.ControlSystemSimulation(sys_beg)
sim_beg.input['Beginning'] = means[2]
sim_beg.compute()
print(np.round(sim_beg.output['output'],3))

0.0
0.0
0.0
